In [1]:
# 1
import numpy as np
import matplotlib.pyplot as plt
from ipyleaflet import Map, basemaps, basemap_to_tiles
import geemap,ee
import shapely
import geopandas as gpd
import json
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipyleaflet import Map, basemaps, WidgetControl
from IPython.display import display,update_display
from datetime import datetime
import threading
import copy
import rasterio as rio
import pandas as pd

In [2]:
# 2
# import ee
ee.Authenticate()

Enter verification code:  4/1AfJohXnN4Hs9JxMunfXmd6Tq9JvEzjJdgJRK8BRTrSnpmTYERWH64dEnX5I



Successfully saved authorization token.


In [3]:
# 3
ee.Initialize()

# [UPDATE ALWAYS]Set land cover classes, and file paths/urls of shapefile and raster file

In [8]:
# 4
classes = ['Annual Crop','Brush/Shrubs','Built-up','Closed Forest','Fishpond','Grassland','Inland Water','Mangrove Forest','Marshland/Swamp','Open Forest','Open/Barren','Perennial Crop','Mixed','Others','Fallow Land', 'Clouds']
shapefile_url = "ESPT1/ESPT2.shp"
raster_url = 'to_segment_naga_RGB.tif'
# new_shapefile_url="ESPT4.geojson"

# [Raster image adjuster - optional] - modify and run if loaded basis raster is not ok

In [9]:
# 5
"function to create bright version of image"
# Reading of raster file, its value, and its properties
with rio.open(raster_url) as ds:
    arr=ds.read()
    print ("dtype",arr.dtype)
    #arr=arr.astype("uint8")
    stretched=np.copy(arr)
    print ("mod dtype",stretched.dtype)
    i_total=arr.shape[1]
    j_total=arr.shape[2]
    b_total=arr.shape[0]
    gamma=0.5
    stack=[]
    for b in range(b_total):
        stack.append(stretched[b][:][:])

    print ("check")
    print (f"1st: {stack[0].shape}")
    print (f"2nd: {stack[1].shape}")
    print (f"3rd: {stack[2].shape}")

    b_stretched=255*((stack[0]/255)**gamma)
    g_stretched=255*((stack[1]/255)**gamma)
    r_stretched=255*((stack[2]/255)**gamma)
    print ("indiv dtype",b_stretched.dtype)
    bgr_stretched=[b_stretched,g_stretched,r_stretched]
    stretched=np.stack(bgr_stretched)
    print (f"stacked shape: {stretched.shape}")              
    print ("array shape:",arr.shape)
    print ("mod dtype",stretched.dtype)
    
data=rio.open(raster_url)
values = data.read([(i+1) for i in range (data.count)])     ##loading bands (create iterative func)
crs = data.crs                                          ##create as one function
transform = data.transform
height = data.height
count=data.count
width = data.width
data.close()

# Defining windows for windowed processing of raster file
new_raster_url="raster_final.tif"
new_dataset = rio.open(new_raster_url, 'w', driver='GTiff',
                            height = height, width = width,
                            count=count, dtype=stretched.dtype,
                            crs=crs,
                            transform=transform)

new_dataset.write(stretched)
new_dataset.close()          

dtype uint16
mod dtype uint16
check
1st: (5804, 9479)
2nd: (5804, 9479)
3rd: (5804, 9479)
indiv dtype float64
stacked shape: (3, 5804, 9479)
array shape: (3, 5804, 9479)
mod dtype float64


# [Vector label maker -optional] - modify and run if vector label container is not compatible

In [12]:
# 6
#function to convert ecog shp to usable
test_shape = gpd.read_file(shapefile_url)
df=gpd.GeoDataFrame(test_shape.copy(deep=False))
#columns=["index","class","classified_by"]
##index,class,classified by, geometry

#df.rename(columns={"ID":"index"},inplace=True)
df.drop(["class","index","classified"],axis=1,inplace=True)
indices=[i for i in range (df.shape[0])]
df.insert(0,"index",indices,True)
classd=[None for i in range (df.shape[0])]
classified=["wala pa" for i in range (df.shape[0])]      #creating columns
df.insert(1,'class',classd,True)
df.insert(2,'classified_by',classified,True)
df=df.astype({"index":int})
df.insert(3,'time_edited',classified,True)
                                                    #putting index
new_shapefile_url="ESPT5.geojson"
df.to_file(new_shapefile_url,index=False,driver="GeoJSON")

In [25]:
# 7
df

,index,class,classified_by,time_edited,geometry
0,0,None,wala pa,wala pa,"MULTIPOLYGON (((123.37657 13.65513, 123.37656 ..."
1,1,None,wala pa,wala pa,"POLYGON ((123.28946 13.67448, 123.29313 13.674..."
2,2,None,wala pa,wala pa,"POLYGON ((123.30394 13.67382, 123.30392 13.673..."
3,3,None,wala pa,wala pa,"POLYGON ((123.30400 13.67447, 123.31487 13.674..."
4,4,None,wala pa,wala pa,"POLYGON ((123.31024 13.67432, 123.31024 13.674..."
...,...,...,...,...,...
9017,9017,None,wala pa,wala pa,"POLYGON ((123.18721 13.60275, 123.18721 13.603..."
9018,9018,None,wala pa,wala pa,"POLYGON ((123.19650 13.60684, 123.19647 13.606..."
9019,9019,None,wala pa,wala pa,"POLYGON ((123.18599 13.60329, 123.18590 13.603..."
9020,9020,None,wala pa,wala pa,"POLYGON ((123.18665 13.60302, 123.18665 13.602..."


# Widget to display maps and labels

In [10]:
# 8
#shapes = gpd.read_file(shapefile_url)
labelled_td = "ESPT5.geojson"
raster_to_use = "to_segment_naga_RGB_for_label.tif"

shapes = gpd.read_file(labelled_td)
print ("worked1")
ind = np.where(pd.isna(shapes['class']))[0][0]
print ("worked2")
coords = list(shapes.loc[ind].geometry.centroid.coords)[0]
Map = geemap.Map(layout=widgets.Layout(width='1200px', height='90px'))#center=(coords[1],coords[0]),zoom=16)
#Map.clear_controls()
out = widgets.Output()
seg_count=len(shapes.index)
labelled_count=seg_count-np.count_nonzero(pd.isnull(shapes), axis=0)[1]

# widg = radio button widget for selecting class
# widg2 = button widget to assign class chosen in widg
# widg3 = button widget to save the shapefile
# widg4 = text widget wherein the name of the labeller can be put
# widg5 = text widget to show current number of labelled

if shapes.loc[ind,'class']==shapes.loc[ind,'class']:
    widg = widgets.RadioButtons(
        options=classes,
        equals = shapes.loc[ind,'class'],
        description='Land cover:',
        layout = widgets.Layout(width='200px'),
    )
else:
    widg = widgets.RadioButtons(
        options=classes,
        description='Land cover:',
        layout = widgets.Layout(width='200px'),
    )

widg_con = WidgetControl(widget=widg,position='topright')

widg2 = widgets.Button(
    description='Set',
    tooltip='Set',
    layout = widgets.Layout(width='100px')
)
widg_con2 = WidgetControl(widget=widg2,position='topright')

widg3 = widgets.Button(
    description='Save File',
    tooltip='Save File',
    layout = widgets.Layout(width='100px')
)
widg_con3 = WidgetControl(widget=widg3,position='topright')

widg4 = widgets.Text(
    value='JAP',
    placeholder='Write your name',
    description='Classified by:',
    disabled=False   
)

widg_con4 = WidgetControl(widget=widg4,position='topleft')


widg5 = widgets.Text(
    value=str(labelled_count)+" out of "+str(seg_count)+" ("+str(round(labelled_count/seg_count*100,2))+") %",
    placeholder='number of labels',
    description='Progress',
    disabled=True,
    continuous_update=True
)

widg_con5 = WidgetControl(widget=widg5,position='topleft')


Map.add_control(control=widg_con5)
def assign(b):                                                                            # Behavior of the "Set" button
    time_edited =datetime.now().strftime("%m/%d/%Y")

    global seg_count,labelled_count,ind, shapes, Map#, newMap
    
    shapes.loc[ind,"class"] = widg.value                      #updateing vector values
    shapes.loc[ind,'classified_by'] = widg4.value
    shapes.loc[ind,'time_edited']=time_edited
    
    
    Map.remove(Map.find_layer("shape"))
    
    ind = ind + 1
    coords = list(shapes.loc[ind].geometry.centroid.coords)[0]
    Map.add_gdf(shapes.loc[ind:ind],layer_name='shape',info_mode=None,fill_colors=["transparent"],style={"color":"#ff0000","weight":5},zoom_to_layer=True)
    Map.setCenter(lon=coords[0],lat=coords[1],zoom=17)
    
    seg_count=len(shapes.index)
    labelled_count=seg_count-np.count_nonzero(pd.isnull(shapes), axis=0)[1]
    widg5.value=str(labelled_count)+" out of "+str(seg_count)+" ("+str(round(labelled_count/seg_count*100,2))+") %"
    
    update_display(Map,display_id='map')
    if shapes.loc[ind,'class']==shapes.loc[ind,'class']:
        print(shapes.loc[ind,'class'])
        print ("what is printing")
        with widg.hold_trait_notifications():
            widg.value = shapes.loc[ind,'class']
        #with widg5.hold_trait_notifications():
        #    widg5.value=str(labelled_count)+" out of "+str(seg_count),
    

def save(b):                                                                            # Behavior of the "Save" button
    global shapes, labelled_td
    shapes.to_file(labelled_td)


widg2.on_click(assign)
widg3.on_click(save)
#widg5.observe(labelled_count)

Map.add_basemap('SATELLITE')
Map.add_raster(new_raster_url,band=[3,2,1],layer_name='sentinel')      ##change this depending on satellite
Map.add_gdf(shapes.loc[ind:ind],layer_name='shape',info_mode=None,zoom_to_layer=True,fill_colors=["transparent"],style={"color":"#ff0000","weight":2})
Map.zoom_to_bounds(shapes.loc[ind].geometry.bounds)
Map.setCenter(lon=coords[0],lat=coords[1],zoom=20)
Map.add_control(control=widg_con)
Map.add_control(control=widg_con2)
Map.add_control(control=widg_con3)
Map.add_control(control=widg_con4)


#Map.addLayerControl()
#Map.add_draw_control()

temp = display(Map,display_id='map')

worked1
worked2


Map(center=[13.663327730812952, 123.26332179341834], controls=(WidgetControl(options=['position', 'transparent…

In [8]:
shapes

,index,class,classified_by,time_edited,geometry
0,0,Others,JAP,09/26/2023,"MULTIPOLYGON (((123.37657 13.65513, 123.37656 ..."
1,1,Others,JAP,09/26/2023,"POLYGON ((123.28946 13.67448, 123.29313 13.674..."
2,2,Clouds,JAP,09/26/2023,"POLYGON ((123.30394 13.67382, 123.30392 13.673..."
3,3,Brush/Shrubs,JAP,09/26/2023,"POLYGON ((123.30400 13.67447, 123.31487 13.674..."
4,4,Brush/Shrubs,JAP,09/26/2023,"POLYGON ((123.31024 13.67432, 123.31024 13.674..."
...,...,...,...,...,...
9017,9017,NaN,wala pa,wala pa,"POLYGON ((123.18721 13.60275, 123.18721 13.603..."
9018,9018,NaN,wala pa,wala pa,"POLYGON ((123.19650 13.60684, 123.19647 13.606..."
9019,9019,NaN,wala pa,wala pa,"POLYGON ((123.18599 13.60329, 123.18590 13.603..."
9020,9020,NaN,wala pa,wala pa,"POLYGON ((123.18665 13.60302, 123.18665 13.602..."


In [14]:
np.count_nonzero(pd.isnull(shapes), axis=0)[1]

8814

In [9]:
len(shapes.index)

9022